## This is hopefully a page to port our query yahoo finance with requests

We're gonna recreate functionality from these two files with requests to cache and hopefully deal with NA's.
https://github.com/lukaszbanasiak/yahoo-finance/tree/master/yahoo_finance

In [4]:
import requests
import requests_cache
import pandas as pd

In [5]:
requests_cache.install_cache('rcache')

In [6]:
# Yahoo! YQL API
PUBLIC_API_URL = 'https://query.yahooapis.com/v1/public/yql'
OAUTH_API_URL = 'https://query.yahooapis.com/v1/yql'
DATATABLES_URL = 'store://datatables.org/alltableswithkeys'

### Hey Audrey you can screencap the below cell and it's output if you want for the presentation

In [7]:
samplereq = requests.get('https://query.yahooapis.com/v1/public/yql', params = {
        'format':'json',
        'env':'store://datatables.org/alltableswithkeys',
        'q':'select * from yaho.finance.historicaldata where symbol = "{}" and startDate = "{}" and endDate = "{}"'.format('YHOO', '2009-09-11', '2010-03-10')
    })
samplereq.url

u'https://query.yahooapis.com/v1/public/yql?env=store%3A%2F%2Fdatatables.org%2Falltableswithkeys&format=json&q=select+%2A+from+yaho.finance.historicaldata+where+symbol+%3D+%22YHOO%22+and+startDate+%3D+%222009-09-11%22+and+endDate+%3D+%222010-03-10%22'

In [8]:
'''
# example of the code from yahoo financefunction
# this will ideally be the part replicated with reqeusts

class YQLQuery(object):

    def execute(self, yql, token=None):
        req = urlopen(PUBLIC_API_URL + '?' + urlencode({
            'q': yql,
            'format': 'json',
            'env': DATATABLES_URL
        }))
        return loads(req.read())
'''

"\n# example of the code from yahoo financefunction\n# this will ideally be the part replicated with reqeusts\n\nclass YQLQuery(object):\n\n    def execute(self, yql, token=None):\n        req = urlopen(PUBLIC_API_URL + '?' + urlencode({\n            'q': yql,\n            'format': 'json',\n            'env': DATATABLES_URL\n        }))\n        return loads(req.read())\n"

In [9]:
'''
# This is the part of the query returning 
    def get_historical(self, start_date, end_date):
        """
        Get Yahoo Finance Stock historical prices
        :param start_date: string date in format '2009-09-11'
        :param end_date: string date in format '2009-09-11'
        :return: list
        """
        hist = []
        for s, e in get_date_range(start_date, end_date):
            try:
                query = self._prepare_query(table='historicaldata', startDate=s, endDate=e)
                result = self._request(query)
                if isinstance(result, dict):
                    result = [result]
                hist.extend(result)
            except AttributeError:
                pass
        return hist
'''

'\n# This is the part of the query returning \n    def get_historical(self, start_date, end_date):\n        """\n        Get Yahoo Finance Stock historical prices\n        :param start_date: string date in format \'2009-09-11\'\n        :param end_date: string date in format \'2009-09-11\'\n        :return: list\n        """\n        hist = []\n        for s, e in get_date_range(start_date, end_date):\n            try:\n                query = self._prepare_query(table=\'historicaldata\', startDate=s, endDate=e)\n                result = self._request(query)\n                if isinstance(result, dict):\n                    result = [result]\n                hist.extend(result)\n            except AttributeError:\n                pass\n        return hist\n'

In [10]:
'''
# This is the part of the code that builds the query
    def _prepare_query(self, table='quotes', key='symbol', **kwargs):
        """
        Simple YQL query bulder
        """
        query = 'select * from yahoo.finance.{table} where {key} = "{symbol}"'.format(
            symbol=self.symbol, table=table, key=key)
        if kwargs:
            query += ''.join(' and {0}="{1}"'.format(k, v)
                             for k, v in kwargs.items())
        return query
'''

'\n# This is the part of the code that builds the query\n    def _prepare_query(self, table=\'quotes\', key=\'symbol\', **kwargs):\n        """\n        Simple YQL query bulder\n        """\n        query = \'select * from yahoo.finance.{table} where {key} = "{symbol}"\'.format(\n            symbol=self.symbol, table=table, key=key)\n        if kwargs:\n            query += \'\'.join(\' and {0}="{1}"\'.format(k, v)\n                             for k, v in kwargs.items())\n        return query\n'

In [11]:
'select * from yaho.finance.historicaldata where symbol = "{}" and startDate = "{}" and endDate = "{}"'.format('YHOO', '2009-09-11', '2010-03-10')

'select * from yaho.finance.historicaldata where symbol = "YHOO" and startDate = "2009-09-11" and endDate = "2010-03-10"'

In [12]:
def myreq(ticker, start, end):
    '''
    input ticker & dates as strings form 'YYYY-MM-DD'
    '''
    params = {'format':'json',
             'env':DATATABLES_URL}
    query = 'select * from yahoo.finance.historicaldata where symbol = "{}" and startDate = "{}" and endDate = "{}"'.format(ticker,start, end)
    params.update({'q':query})
    req = requests.get(PUBLIC_API_URL, params=params)
    req.raise_for_status()
    req = req.json()
    if req['query']['count'] > 0:
        result = req['query']['results']['quote']
        return result
    else:
        pass

In [13]:
years = [2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]

In [14]:
y = years[3]

In [48]:
yreq = myreq('PIH','{}-01-01'.format(y),'{}-12-31'.format(y))
yreq is None

True

In [42]:
csv = pd.read_csv('./companylist.csv')
# We want to keep "Finance, Health Care, Technology, Energy"
newcsv = csv[csv["Sector"].isin(["Finance", "Energy","Health Care","Technology"])].reset_index()
del newcsv["index"]
whole_list = newcsv['Symbol']
# whole_list

In [51]:
#NEW price function
def price2(ticker):
    date=[]
    price=[]
    report = []
    
    years = [2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]
    for x in range(len(years)):
        c = myreq(ticker,'{}-01-01'.format(years[x]),'{}-12-31'.format(years[x]))
        try:
        
            for i in range(0,len(c)):
                date.append(pd.to_datetime(c[i]["Date"]))
                price.append(float(c[i][u'Close']))
                datef = pd.DataFrame(date)
                pricef = pd.DataFrame(price)
                table1 = pd.concat([datef,pricef],axis = 1)
                table1.columns = ['Date', ticker]
                table1 = table1.set_index("Date")
            
        except Exception:
            table1 = pd.DataFrame()
    
    return table1

In [54]:
#del df
for l in range(10,20):
    print whole_list[l]
    get = price2(whole_list[l])
    try:
        df = pd.concat([df,get],axis = 1)    # concat. by column 
    except NameError:
        df = pd.DataFrame(get)    # initialize automatically
df.shape
df

AXAS
ACIU
ACIA
ACHC
ACAD
ACST
XLRN
ANCX
ARAY
ACRX


,PIH,FCCY,SRCE,VNET,TWOU,JOBS,ABEO,ABIL,ABMD,AXAS,ACIU,ACIA,ACHC,ACAD,ACST,XLRN,ANCX,ARAY,ACRX
Date,,,,,,,,,,,,,,,,,,,
2006-01-03,NaN,21.499973,25.830002,NaN,NaN,14.950000,0.51,NaN,9.350000,5.55,NaN,NaN,2.050000,9.990000,NaN,NaN,14.940000,NaN,NaN
2006-01-04,NaN,21.499973,25.659998,NaN,NaN,14.790000,0.47,NaN,9.620000,5.55,NaN,NaN,2.060000,9.970000,NaN,NaN,14.610000,NaN,NaN
2006-01-05,NaN,20.999980,25.820004,NaN,NaN,16.150000,0.46,NaN,9.550000,5.81,NaN,NaN,2.100000,10.010000,NaN,NaN,14.520000,NaN,NaN
2006-01-06,NaN,20.519969,25.950002,NaN,NaN,17.080000,0.45,NaN,9.750000,5.95,NaN,NaN,2.180000,10.460000,NaN,NaN,13.470000,NaN,NaN
2006-01-09,NaN,20.249976,25.999997,NaN,NaN,17.900000,0.43,NaN,10.150000,5.55,NaN,NaN,2.200000,10.690000,NaN,NaN,11.300000,NaN,NaN
2006-01-10,NaN,19.999980,25.999997,NaN,NaN,17.860001,0.46,NaN,10.210000,5.69,NaN,NaN,2.280000,10.990000,NaN,NaN,11.250000,NaN,NaN
2006-01-11,NaN,20.369968,25.999997,NaN,NaN,17.950001,0.50,NaN,10.550000,5.50,NaN,NaN,2.280000,10.800000,NaN,NaN,10.400000,NaN,NaN
2006-01-12,NaN,20.000001,25.940002,NaN,NaN,17.480000,0.48,NaN,10.710000,5.46,NaN,NaN,2.310000,10.710000,NaN,NaN,10.570000,NaN,NaN
2006-01-13,NaN,20.779999,25.989997,NaN,NaN,17.809999,0.44,NaN,10.270000,5.42,NaN,NaN,2.260000,10.710000,NaN,NaN,10.550000,NaN,NaN
